In [1]:
import numpy as np
import pickle

In [46]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_save = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save"
path_tables = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/tables"
path_views = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/views"
path_chkpt = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint"

In [27]:
batchsize = 32
visitfile = np.array(pickle.load(open(path_save + '/3digitICD9.seqs', 'rb')))
labelfile = np.array(pickle.load(open(path_save + '/morts', 'rb')))
timeFile = np.array(pickle.load(open(path_save + '/dates', 'rb')))
batchsize = 32
dropoutrate = 0.2
EPOCHIN = 10
ATTENTION_DIM = 8
HIDDEN_DIM = 64
EMBEDDING_DIM = 64

C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [28]:
print(timeFile)

[list([datetime.datetime(2153, 9, 3, 7, 15), datetime.datetime(2157, 10, 18, 19, 34)])
 list([datetime.datetime(2186, 7, 18, 16, 46), datetime.datetime(2191, 2, 23, 5, 23)])
 list([datetime.datetime(2131, 4, 30, 7, 15), datetime.datetime(2131, 5, 12, 19, 49), datetime.datetime(2134, 5, 10, 11, 30)])
 ...
 list([datetime.datetime(2127, 2, 24, 7, 15), datetime.datetime(2127, 3, 25, 16, 52), datetime.datetime(2127, 6, 10, 19, 22), datetime.datetime(2127, 6, 27, 13, 30), datetime.datetime(2127, 6, 30, 20, 19), datetime.datetime(2127, 10, 21, 12, 43), datetime.datetime(2128, 1, 23, 7, 45)])
 list([datetime.datetime(2165, 5, 5, 20, 56), datetime.datetime(2165, 6, 5, 11, 30)])
 list([datetime.datetime(2186, 1, 16, 10, 30), datetime.datetime(2186, 2, 8, 12, 30), datetime.datetime(2189, 7, 10, 18, 8)])]


In [19]:
data=[]
for i in range(0,len(visitfile)):
    data.append((visitfile[i], labelfile[i]))
        
data=np.array(data)   

([[0, 1, 2, 3, 4, 5, 6, 7], [8, 0, 9, 2, 10, 11, 12, 4, 7]], 0)
([[13, 9, 14, 3, 15, 16, 17, 18], [19, 20, 3, 15, 21, 17, 22, 23]], 0)
([[1, 24, 3, 25, 26, 6, 27, 28, 7], [24, 3, 29, 30, 26, 4, 6, 31, 32, 28, 33, 7], [34, 35, 36, 3, 37, 30, 26, 6, 28, 7]], 0)
([[38, 19, 39, 2, 40, 41, 42, 15, 43, 44, 45, 46], [38, 47, 39, 2, 48, 49, 42, 50, 51, 52, 53, 44, 45, 23, 54, 46, 7], [24, 48, 42, 45, 38, 3, 15, 55, 23, 56, 7, 49, 29, 16, 57, 22, 52, 58, 59, 47, 2, 60, 61], [62, 38, 39, 60, 48, 49, 3, 15, 55, 22, 51, 23, 45, 63, 7], [34, 64, 48, 42, 65, 66, 62, 38, 67, 39, 3, 15, 55, 23, 7, 68, 49, 22, 1, 2, 60]], 1)
([[59, 69, 70, 71, 2, 72, 49, 55, 73, 74], [7, 75, 1, 2, 20, 76, 72, 29, 77, 32, 31, 78, 79]], 0)
([[7, 13, 25, 29, 80, 51, 18, 81], [0, 15, 80, 18, 7, 81]], 0)
([[34, 14, 24, 49, 6, 66, 82, 68], [34, 59, 69, 14, 24, 3, 42, 55, 15, 6, 83, 66, 84, 82, 68]], 0)
([[85, 67, 86, 87, 88, 72, 61, 51, 89], [67, 87, 90, 91, 48, 49, 55, 92, 68, 18, 51, 74, 52, 56, 93]], 1)
([[94, 72, 55, 23,

([[2, 96, 11, 45, 7], [42, 73, 284, 45, 226, 54, 7, 20, 69, 49, 16, 50, 52, 32, 336, 63, 322, 190, 1, 60, 397, 162, 119]], 0)
([[528, 10, 186, 287, 413, 95], [528, 153, 10, 55, 21, 175], [2, 528, 24, 72, 186, 108, 413, 164, 336, 45]], 0)
([[222, 2, 281, 619, 27, 118, 316], [118]], 0)
([[1, 151, 241, 347, 42, 15, 6, 27], [107, 69, 200, 72, 49, 412, 15, 295, 65, 116, 22, 58, 23, 198, 199]], 1)
([[3, 15, 231, 17, 101, 23, 95, 7], [0, 270, 273, 15, 231, 4, 98, 57, 7, 374], [2, 273, 3, 15, 4, 57, 98, 316]], 0)
([[258, 281, 260, 261, 214, 118], [118, 189]], 0)
([[4, 28, 145, 39, 3, 15, 55, 77, 142, 23, 18, 612, 210, 198, 7, 99, 332, 49, 50, 52, 32, 112, 243, 199, 1, 35], [19, 72, 42, 4, 51, 28, 15, 55, 77, 23, 18, 198, 88, 49, 139, 238, 199, 1, 244, 60]], 1)
([[53, 0, 9, 3, 15, 55, 18, 45, 7], [393, 522, 9, 3, 15, 18, 162, 53, 7]], 0)
([[13, 88, 254, 3, 139, 238, 18, 212, 235], [19, 69, 2, 14, 72, 49, 273, 15, 63, 4, 382, 22, 52, 45, 118, 165, 56], [86, 19, 1, 2, 204, 72, 49, 84, 4, 162, 118

([[88, 35, 273, 245, 3, 55, 45, 243, 7], [103, 96, 72, 55, 4, 51, 95, 45, 63, 7]], 0)
([[59, 13, 2, 270, 3, 55, 246, 73, 108, 110, 7, 68, 95], [24, 72, 64, 138, 4, 38, 3, 15, 55, 108, 142, 105, 7, 68, 402, 49, 41, 22, 58, 95, 63, 35, 2, 241, 61, 346, 231, 289]], 1)
([[88, 72, 25, 15, 73, 295, 504, 105, 7], [13, 20, 40, 49, 181, 30, 18, 53, 119, 145, 105, 93]], 0)
([[1, 20, 2, 3, 16, 17, 51], [152, 3, 72, 15, 16, 22, 31]], 0)
([[34, 88, 2, 72, 55, 23, 68], [70, 2, 266, 42, 23, 95]], 0)
([[34, 20, 3, 49, 15, 4, 22, 18], [69, 72, 49, 3, 55, 4, 170, 51, 18]], 1)
([[208, 27, 222], [118]], 0)
([[69, 72, 3, 193, 77, 124], [13, 264, 3, 29, 181, 73, 77, 280, 287, 4, 51, 18, 95, 118], [86, 24, 138, 73, 280, 677, 4, 51, 635, 15, 181, 77, 203, 18, 118, 7, 88, 49, 22, 95, 121, 238, 426, 539]], 0)
([[1, 2, 3, 271, 45, 7], [664, 112, 34, 7, 47, 72, 49, 55, 355, 52, 210, 212, 45, 197, 147, 56, 46]], 0)
([[169, 0, 69, 132, 15, 124, 147, 63, 238], [56, 34, 191, 200, 48, 42, 124, 63, 68]], 1)
([[34, 281,

([[13, 114, 152, 3, 266, 181, 73, 7], [209, 82, 308, 15, 23, 419, 417, 120, 114, 12, 6, 269, 135, 127, 156, 2, 11, 386, 165]], 1)
([[38, 127, 215, 72, 41, 214, 119, 301, 7], [38, 127, 69, 2, 72, 236, 41, 216, 55, 27, 301, 7], [64, 72, 42, 158, 124, 436, 28, 301, 38, 215, 15, 55, 289, 159, 101, 89, 56, 7, 99, 90, 49, 41, 52, 127, 1, 47, 2, 184, 94]], 1)
([[8, 19, 88, 270, 166, 152, 72, 607, 55, 21, 159, 314, 129, 45, 126], [8, 69, 270, 72, 174, 21, 51, 129, 45, 63], [275, 270, 69, 166, 24, 72, 437, 65, 492, 582, 6, 129], [19, 72, 148, 21, 84, 3, 15, 55, 198, 437, 139, 694, 80, 22, 95, 129, 238, 199, 8, 492]], 0)
([[7, 2, 14, 24, 3, 15, 55, 18, 159, 22, 53, 330], [0, 2, 24, 3, 40, 72, 73, 4, 32, 18, 480, 7]], 0)
([[120, 464, 2, 0, 3, 49, 15, 51, 18, 95, 147, 7], [24, 72, 42, 51, 178, 62, 464, 3, 15, 55, 18, 120, 69, 49, 47, 2, 60, 220, 170]], 1)
([[433, 2, 208, 3, 281, 25, 266, 16, 345, 118, 137, 7], [122, 2, 24, 273, 49, 29, 16, 345, 31, 269, 137], [2, 24, 3, 137, 7]], 0)
([[19, 73, 116

C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [21]:
data.shape

(7537, 2)

In [22]:
code_to_ix = {}
label_to_ix={}
ix_to_code={}
ix_to_code[0]='OUT'
    
for visits, label in data:
    for visit in visits:
        for code in visit:
            if code not in code_to_ix:
                code_to_ix[code] = len(code_to_ix)+1
                ix_to_code[code_to_ix[code]]=code  
        
    if label not in label_to_ix:
        label_to_ix[label]=len(label_to_ix)

In [23]:
lenlist=[]
for i in data:
    lenlist.append(len(i[0]))
sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
new_data=data[sortlen]

In [24]:
batch_data=[]

In [48]:
for start_ix in range(0,1):# len(new_data)-batchsize+1, batchsize):
    thisblock=new_data[start_ix:start_ix+batchsize]
    mybsize= len(thisblock)
    mynumvisit=np.max([len(ii[0]) for ii in thisblock])
    mynumcode=np.max([len(jj) for ii in thisblock for jj in ii[0] ])
    main_matrix = np.zeros((mybsize, mynumvisit, mynumcode), dtype= np.int)
    mask_matrix = np.zeros((mybsize, mynumvisit, mynumcode), dtype= np.float32)
    mylabel=[]
    for i in range(main_matrix.shape[0]):
        for j in range(main_matrix.shape[1]):
                try:
                    main_matrix[i,j] = code_to_ix[thisblock[i][0][j][k]]
                        
                except IndexError:
                    mask_matrix[i,j,k] = 1e+20

batch_data.append(((autograd.Variable(torch.from_numpy(main_matrix)), autograd.Variable(torch.from_numpy(mask_matrix)), autograd.Variable(torch.LongTensor(mylabel)))))

C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [90]:
class MRM(nn.Module):

    def __init__(self, batch_size, embedding_dim, hidden_dim, attention_dim, vocab_size, labelset_size,dropoutrate):
        super(MRM, self).__init__()
        self.hidden_dim = hidden_dim
        self.batchsi=batch_size
        self.word_embeddings = nn.Embedding(vocab_size+1, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2, labelset_size)
        self.hidden = self.init_hidden()
        self.attention=nn.Linear(embedding_dim, attention_dim)
        self.vector1=nn.Parameter(torch.randn(attention_dim,1))
        self.decay=nn.Parameter(torch.FloatTensor([-0.1]*(vocab_size+1)))    
        self.initial=nn.Parameter(torch.FloatTensor([1.0]*(vocab_size+1)))   
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.attention_dimensionality=attention_dim
        self.WQ1=nn.Linear(embedding_dim, attention_dim,bias=False)
        self.WK1=nn.Linear(embedding_dim, attention_dim,bias=False)
        self.embed_drop = nn.Dropout(p=dropoutrate)
        

    def init_hidden(self):
        
        return (autograd.Variable(torch.zeros(2, self.batchsi, self.hidden_dim)),
                autograd.Variable(torch.zeros(2, self.batchsi, self.hidden_dim)))

    def forward(self, sentence, Mode):
        numcode=sentence[0].size()[1]
        numvisit=sentence[0].size()[0]
        numbatch=1#sentence[0].size()[0]
        thisembeddings =self.word_embeddings(sentence[0].view(-1,numcode))
        thisembeddings = self.embed_drop(thisembeddings)
        myQ1=self.WQ1(thisembeddings)
        myK1=self.WK1(thisembeddings)
        dproduct1= torch.bmm(myQ1, torch.transpose(myK1,1,2)).view(numbatch,numvisit,numcode,numcode)
        dproduct1=dproduct1-sentence[1].view(numbatch,numvisit,1,numcode)-sentence[1].view(numbatch,numvisit,numcode,1)
        sproduct1=self.softmax(dproduct1.view(-1,numcode)/np.sqrt(self.attention_dimensionality)).view(-1,numcode,numcode) 
        fembedding11=torch.bmm(sproduct1,thisembeddings)
        fembedding11=(((sentence[1]-(1e+20))/(-1e+20)).view(-1,numcode,1)*fembedding11)
        mydecay = self.decay[sentence[0].view(-1)].view(numvisit*numbatch,numcode,1)
        myini = self.initial[sentence[0].view(-1)].view(numvisit*numbatch, numcode,1)
        temp1= torch.bmm( mydecay, sentence[2].view(-1,1,1))
        temp2 = self.sigmoid(temp1+myini)   
        vv=torch.bmm(temp2.view(-1,1,numcode),fembedding11)
        vv=vv.view(numbatch,numvisit,-1).transpose(0,1)
        lstm_out, self.hidden = self.lstm(vv, self.hidden)
        label_space = self.hidden2label(lstm_out[-1])
        label_scores = self.softmax(label_space)
        return label_scores

In [91]:
model = MRM(batchsize, EMBEDDING_DIM, HIDDEN_DIM, ATTENTION_DIM, len(code_to_ix), len(label_to_ix),dropoutrate)
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())


In [92]:
ep=0
while ep <5:  
    model.train()
    for mysentence in batch_data:
        print(mysentence[0][0])
        print(mysentence[0][1])
        print(mysentence[0][2])
        t = mysentence[0][2]
        print(t.size())
        model.zero_grad()
        model.hidden = model.init_hidden()
        targets = mysentence[1]
        label_scores = model((mysentence[0][0],mysentence[0][1],mysentence[0][2]), 1)
        loss = loss_function(label_scores, targets)
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), 'model_epoch_'+str(ep))         
    print ('finished', ep, 'epochs')
    print ('on validation set:')
    model.eval()
    model.hidden = model.init_hidden()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32)
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32)
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32)
torch.Size([2, 23])


C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


IndexError: tensors used as indices must be long, byte or bool tensors

In [ ]:
#Second Attempt to implement

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
"""
Attention Layer 
"""
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        s= [x for x in NE if x > topk]  
        alpha = K.softmax(s)
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
"""
Event pooling layer 
"""
class EventPooling(Layer):

def __init__(self, k=1, **kwargs):
    super().__init__(**kwargs)
    self.input_spec = InputSpec(ndim=3)
    self.k = k

def compute_output_shape(self, input_shape):
    return (input_shape[0], (input_shape[2] * self.k))

def call(self, inputs):
    
    shifted_input = tf.transpose(inputs, [0, 2, 1])
    top_k = tf.nn.top_k(shifted_input, k=self.k, sorted=True, name=None)[0]
    
    # return flattened output
    return Flatten()(top_k)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add((LSTM(64, return_sequences=True)))
model.add(attention(return_sequences=True)) 
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

In [ ]:
def MRM_model(input_shape, hidden = 256, targets = 1, learn_rate = 1e-4):
    model = Sequential()
    model.add(Bidirectional(LSTM(hidden), merge_mode = 'concat'))
    model.add(Activation ('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(targets))
    model.compile(loss='binary_crossentropy', 
    optimizer=Adam(lr=learn_rate, beta_1 =.25 ), metrics=['accuracy'])
    return (model)

In [ ]:
MRM_model = MRM_model(input_shape=input_shape, targets=1, hidden = 256, learn_rate=1e-4)

In [ ]:
MRM_model.fit(x = X, y= y, validation_split=.2, epochs = 10, verbose=2, batch_size=32, callbacks=[checkpoint])